In [ ]:
import numpy as np
import pandas as pd 
from numpy import sqrt, abs, round
import seaborn as sns
from scipy.stats import norm, uniform
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('../input/students-performance-in-exams/StudentsPerformance.csv')

# Students Performance:

In [ ]:
data.head()

# Now We Want To Sum math, reading, writing into totalScore

In [ ]:
data['total'] = data['math score'] + data['reading score'] + data['writing score']

In [ ]:
data.drop(['math score', 'reading score', 'writing score'], axis=1, inplace=True)

In [ ]:
data.rename(columns={'race/ethnicity': 'group'}, inplace=True)

In [ ]:
data['group'].value_counts()

**see we have A, B, C, D, E group Now We Want To See Mean Of Score Each group**

In [ ]:
data.groupby('group').mean()['total']

*Let's See groupE Is More Cleaver Than Other Group or Not*
### We Use Statistic-test For These

In [ ]:
data['total'].mean()

In [ ]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [ ]:
res = ols('total ~ group', data=data).fit()

In [ ]:
sm.stats.anova_lm(res)

## What is df_group: Degree Fredoom (n-1) ==> (5-1) = 4
## What is df_residual: alldata - allGroup ==> 1000 - 5 = 995****

## group_sumsq : allelement - mean 
## residual_sumsq : every element determine specific group [every element - specific group_mean

## group_meansq : group_df / group_sumsq 
## residual_meansq : residual_df / residual_sumsq


## f : group_meansq / residual_meansq
## p_value:cdf(f)

## z_test: 

In [ ]:
dataA = data[data['total'] < data['total'].mean()]['total']
dataB = data[data['total'] >= data['total'].mean()]['total']

In [ ]:
A_mean = dataA.mean()
B_mean = dataB.mean()

In [ ]:
A_std = dataA.std()
B_std = dataB.std()

In [ ]:
A_len = len(dataA)
B_len = len(dataB)

In [ ]:
under = sqrt((A_std ** 2 / A_len) + (B_std ** 2 / B_len))

In [ ]:
above = A_mean - B_mean

In [ ]:
z = above / under
p_value = 2 * (1-norm.cdf(abs(z)))

In [ ]:
p_value
### what does it mean it mean some reason exist between class and classE is clever than the other ###

In [ ]:
data.groupby(['group', 'parental level of education']).size().plot(kind='barh', figsize=(10, 10))

In [ ]:
data.groupby(['group', 'test preparation course']).size().plot(kind='barh', figsize=(10, 10), grid=True)

In [ ]:
def level_edu(columns):
    if columns == "associate's degree":
        return np.nan
    elif columns == "some college":
        return np.nan
    elif columns == "some high school":
        return np.nan
    elif columns == 'high school':
        return np.nan
    else:
        return columns

## Run This :

In [ ]:
#data['parental level of education'] = data['parental level of education'].map(level_edu)

In [ ]:
delet_data = data.dropna()

In [ ]:
delet_data.groupby('group').mean()

In [ ]:
delet_data.groupby(['group', 'parental level of education', 'gender', 'test preparation course']).size()

In [ ]:
sns.distplot(data.select_dtypes(include=['int64']))

## If You Want Find Parameter For Distribution Use Norm  or Possion or Uniform .fit(your-data)

In [ ]:
norm.fit(data.select_dtypes(include=['int64']))
#mean = 203, std = 42

In [ ]:
## just example
#uniform.fit(data.select_dtypes(include=['int64']))

In [ ]:
data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
x = data.loc[:, ['group', 'gender', 'parental level of education', 'lunch', 'test preparation course', 'total']]
x['total'] = MinMaxScaler().fit_transform(x['total'].values.reshape(-1, 1))


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer

In [ ]:
label = LabelEncoder()
objList = data.select_dtypes(include = "object").columns
for column in objList:
    x[column] = label.fit_transform(x[column].astype(str))
y = x['group']

### select KBest, Chi2: 

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
bestfeatures = SelectKBest(score_func=chi2, k=2)
fit = bestfeatures.fit(x, y)
score = pd.DataFrame(fit.scores_)
columns = pd.DataFrame(x.columns)
featureScores = pd.concat([columns,score],axis=1)
featureScores.columns = ['columns','Score']
featureScores

### plot Scatter Clustering:

In [ ]:
from sklearn import cluster
w2 = cluster.AgglomerativeClustering(n_clusters=2, linkage='ward').fit(x)
w3 = cluster.AgglomerativeClustering(n_clusters=3, linkage='ward').fit(x)
w4 = cluster.AgglomerativeClustering(n_clusters=4, linkage='ward').fit(x)

In [ ]:
plt.figure(figsize=(10, 10))

plt.subplot(131)
plt.scatter(x.iloc[:, -1], x.iloc[:, 2], c=w2.labels_)
plt.title("K=2")

plt.subplot(132)
plt.scatter(x.iloc[:, -1], x.iloc[:, 2], c=w3.labels_)
plt.title("K=3")

plt.subplot(133)
plt.scatter(x.iloc[:, -1], x.iloc[:, 2], c=w4.labels_) 
plt.title("K=4")

In [ ]:
x = x.drop('group', axis=1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer


col_transformer = make_column_transformer(
        (OneHotEncoder(), list(x.select_dtypes(include='object'))),
        remainder='passthrough')

x_ohe = col_transformer.fit_transform(x)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
kmean = KMeans(n_clusters=3,  max_iter = 700,n_init = 10 )
y_predict = kmean.fit_predict(x_ohe)

In [ ]:

y = LabelEncoder.fit_transform(x, y)

## How Choose Best KElbow:

In [ ]:
sse = []
for k in range(1, 11):
     kmeans = KMeans(n_clusters=k, max_iter = 300)
     kmeans.fit(x_ohe)
     sse.append(kmeans.inertia_)
        
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
y[:10]

In [ ]:
y_predict[:10]